In [8]:
#Обучение модели и сохраннение ее
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from catboost import CatBoostRegressor
import optuna

# Глобальные переменные для хранения состояния
label_encoders = {}
scalers = {}
model = None
feature_columns = []

# Функция для перевода баллов SAT в баллы ЕГЭ
def sat_to_ege(sat_score):
    """
    Переводит баллы SAT (400–1600) в баллы ЕГЭ (40–100).
    """
    sat_min, sat_max = 400, 1600
    ege_min, ege_max = 40, 100
    if not (sat_min <= sat_score <= sat_max):
        raise ValueError(f"Балл SAT должен быть в диапазоне {sat_min}–{sat_max}.")
    ege_score = ege_min + (sat_score - sat_min) * (ege_max - ege_min) / (sat_max - sat_min)
    return round(ege_score, 2)

def load_and_prepare_data(data_path):
    global label_encoders, scalers, feature_columns
    
    # Загрузка данных
    df = pd.read_csv(data_path)
    
    # Преобразование баллов SAT в баллы ЕГЭ
    df['Балл ЕГЭ'] = df['Балл SAT'].apply(sat_to_ege)
    df.drop(columns=['Балл SAT'], inplace=True)  # Удаляем старый столбец "Балл SAT"
    
    # Кодирование категориальных переменных
    categorical_columns = ['Пол', 'Область изучения', 'Текущий уровень работы', 'Предпринимательство']
    for col in categorical_columns:
        le = LabelEncoder()
        le.fit(df[col].unique())
        df[col] = le.transform(df[col])
        label_encoders[col] = le
    
    # Исключение столбцов с кластерами
    cluster_columns = [col for col in df.columns if '_cluster' in col or col == 'Cluster']
    X = df.drop(columns=['Идентификатор студента', 'Начальная зарплата', 'Годы до повышения', 
                         'Удовлетворенность карьерой', 'Баланс между работой и личной жизнью'] + cluster_columns)
    feature_columns = X.columns.tolist()  # Сохраняем только релевантные признаки
    y = df[['Начальная зарплата', 'Годы до повышения', 'Удовлетворенность карьерой', 'Баланс между работой и личной жизнью']]
    
    # Нормализация признаков
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    scalers['X'] = scaler_X
    
    # Масштабирование целевых переменных
    target_scalers = {
        'salary': MinMaxScaler(feature_range=(25000, 150000)),
        'promotion': MinMaxScaler(feature_range=(1, 5)),
        'satisfaction': MinMaxScaler(feature_range=(1, 10)),
        'balance': MinMaxScaler(feature_range=(1, 10))
    }
    
    y_scaled = np.hstack((
        target_scalers['salary'].fit_transform(y[['Начальная зарплата']]),
        target_scalers['promotion'].fit_transform(y[['Годы до повышения']]),
        target_scalers['satisfaction'].fit_transform(y[['Удовлетворенность карьерой']]),
        target_scalers['balance'].fit_transform(y[['Баланс между работой и личной жизнью']])
    ))
    
    scalers.update(target_scalers)
    
    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=0.4, random_state=42
    )
    
    return X_train, X_test, y_train, y_test

def optimize_hyperparameters(X_train, y_train):
    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 500, 2000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'depth': trial.suggest_int('depth', 4, 10),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
            'random_strength': trial.suggest_float('random_strength', 0.1, 10),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'verbose': 0
        }
        
        model = CatBoostRegressor(**params, loss_function='MultiRMSE')
        
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        scores = []
        
        for train_idx, val_idx in kf.split(X_train):
            X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
            y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
            
            model.fit(X_train_fold, y_train_fold, eval_set=(X_val_fold, y_val_fold), verbose=0)
            predictions = model.predict(X_val_fold)
            error = np.sqrt(np.mean((predictions - y_val_fold) ** 2))
            scores.append(error)
        
        return np.mean(scores)
    
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=15)
    
    print(f"Best hyperparameters: {study.best_params}")
    print(f"Best error value: {study.best_value}")
    
    best_model = CatBoostRegressor(**study.best_params, loss_function='MultiRMSE', verbose=0)
    best_model.fit(X_train, y_train)
    return best_model

def initialize_system(data_path, model_save_path="model.cbm"):
    global model
    X_train, _, y_train, _ = load_and_prepare_data(data_path)
    model = optimize_hyperparameters(X_train, y_train)
    
    # Сохранение модели в файл
    model.save_model(model_save_path)
    print(f"Модель сохранена в файл: {model_save_path}")

def load_model(model_load_path="model.cbm"):
    """
    Загружает модель из файла.
    """
    global model
    model = CatBoostRegressor()
    model.load_model(model_load_path)
    print(f"Модель загружена из файла: {model_load_path}")

def predict(age, gender, hs_gpa, exam_score, exam_type, uni_rank, uni_gpa, field_of_study, 
           internships, projects, certifications, soft_skills, networking, 
           job_offers, current_job_level, entrepreneurship):
    # Преобразование баллов SAT в баллы ЕГЭ, если выбран SAT
    if exam_type == 'SAT':
        ege_score = sat_to_ege(exam_score)
    elif exam_type == 'ЕГЭ':
        ege_score = exam_score
    else:
        raise ValueError("Тип аттестации должен быть 'SAT' или 'ЕГЭ'.")
    
    # Кодирование входных данных
    input_data = {
        'Возраст': age,
        'Пол': label_encoders['Пол'].transform([gender])[0],
        'Средний балл в школе': hs_gpa,
        'Балл ЕГЭ': ege_score,  # Используем баллы ЕГЭ
        'Рейтинг университета': uni_rank,
        'Средний балл в университете': uni_gpa,
        'Область изучения': label_encoders['Область изучения'].transform([field_of_study])[0],
        'Количество стажировок': internships,
        'Количество проектов': projects,
        'Сертификаты': certifications,
        'Оценка мягких навыков': soft_skills,
        'Оценка сетевого взаимодействия': networking,
        'Предложения о работе': job_offers,
        'Текущий уровень работы': label_encoders['Текущий уровень работы'].transform([current_job_level])[0],
        'Предпринимательство': label_encoders['Предпринимательство'].transform([entrepreneurship])[0]
    }
    
    # Создание входного вектора в правильном порядке
    X_new = np.array([[input_data[col] for col in feature_columns]])
    
    # Нормализация и предсказание
    X_scaled = scalers['X'].transform(X_new)
    predictions = model.predict(X_scaled)
    
    # Обратное преобразование результатов
    salary = scalers['salary'].inverse_transform(predictions[:, 0].reshape(-1, 1))[0][0]
    promotion = scalers['promotion'].inverse_transform(predictions[:, 1].reshape(-1, 1))[0][0]
    satisfaction = scalers['satisfaction'].inverse_transform(predictions[:, 2].reshape(-1, 1))[0][0]
    balance = scalers['balance'].inverse_transform(predictions[:, 3].reshape(-1, 1))[0][0]
    
    # Проверка и ограничение значений в допустимых границах
    salary = max(25000, min(150000, salary))  # Ограничение зарплаты
    promotion = max(1, min(5, promotion))    # Ограничение лет до повышения
    satisfaction = max(1, min(10, satisfaction))  # Ограничение удовлетворенности
    balance = max(1, min(10, balance))       # Ограничение баланса
    
    return {
        'salary': salary,
        'promotion': promotion,
        'satisfaction': satisfaction,
        'balance': balance
    }

if __name__ == "__main__":
    initialize_system('education_career_success_translated.csv', model_save_path="model.cbm")
    
    # Предсказание параметров
    result = predict(
        age=25,
        gender='Male',
        hs_gpa=3.8,
        exam_score=1400,  # Балл SAT или ЕГЭ
        exam_type='SAT',  # Тип аттестации: 'SAT' или 'ЕГЭ'
        uni_rank=5,
        uni_gpa=3.5,
        field_of_study='Computer Science',
        internships=2,
        projects=5,
        certifications=3,
        soft_skills=8,
        networking=7,
        job_offers=1,
        current_job_level='Entry',
        entrepreneurship='No'
    )
    
    print("Predicted Values:")
    print(f"Starting Salary: ${result['salary']:.2f}")
    print(f"Years to Promotion: {result['promotion']:.1f}")
    print(f"Career Satisfaction: {result['satisfaction']:.1f}/10")
    print(f"Work-Life Balance: {result['balance']:.1f}/10")

[I 2025-03-19 11:54:01,802] A new study created in memory with name: no-name-8d0c5155-4ee4-40b3-aa58-77d6d88fbc75
[I 2025-03-19 11:54:30,677] Trial 0 finished with value: 12001.230242157004 and parameters: {'iterations': 1108, 'learning_rate': 0.16224475347932554, 'depth': 6, 'l2_leaf_reg': 8.425214049407673, 'random_strength': 5.896072212638382, 'bagging_temperature': 0.7898198078398365}. Best is trial 0 with value: 12001.230242157004.
[I 2025-03-19 11:54:41,013] Trial 1 finished with value: 12002.518179876928 and parameters: {'iterations': 666, 'learning_rate': 0.2820284045617248, 'depth': 5, 'l2_leaf_reg': 9.529529775018796, 'random_strength': 9.2582077881618, 'bagging_temperature': 0.011949035796572915}. Best is trial 0 with value: 12001.230242157004.
[I 2025-03-19 11:59:15,063] Trial 2 finished with value: 12000.912414921166 and parameters: {'iterations': 1962, 'learning_rate': 0.199487444784518, 'depth': 9, 'l2_leaf_reg': 5.295449000472892, 'random_strength': 0.5321887951850858, 

Best hyperparameters: {'iterations': 1405, 'learning_rate': 0.016058177451357784, 'depth': 10, 'l2_leaf_reg': 1.016268128918038, 'random_strength': 3.1239710531524683, 'bagging_temperature': 0.20860745871797912}
Best error value: 11990.04918564373
Модель сохранена в файл: model.cbm
Predicted Values:
Starting Salary: $51155.51
Years to Promotion: 3.1
Career Satisfaction: 6.3/10
Work-Life Balance: 5.8/10


C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [12]:
#ЗАГРУЗКА МОДЕЛИ И ПРЕДСКАЗЫВАНИЕ
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from catboost import CatBoostRegressor

# Глобальные переменные для хранения состояния
label_encoders = {}
scalers = {}
model = None
feature_columns = []

# Функция для перевода баллов SAT в баллы ЕГЭ
def sat_to_ege(sat_score):
    """
    Переводит баллы SAT (400–1600) в баллы ЕГЭ (40–100).
    """
    sat_min, sat_max = 400, 1600
    ege_min, ege_max = 40, 100
    if not (sat_min <= sat_score <= sat_max):
        raise ValueError(f"Балл SAT должен быть в диапазоне {sat_min}–{sat_max}.")
    ege_score = ege_min + (sat_score - sat_min) * (ege_max - ege_min) / (sat_max - sat_min)
    return round(ege_score, 2)

def load_and_prepare_data(data_path):
    global label_encoders, scalers, feature_columns
    
    # Загрузка данных
    df = pd.read_csv(data_path)
    
    # Преобразование баллов SAT в баллы ЕГЭ
    df['Балл ЕГЭ'] = df['Балл SAT'].apply(sat_to_ege)
    df.drop(columns=['Балл SAT'], inplace=True)  # Удаляем старый столбец "Балл SAT"
    
    # Кодирование категориальных переменных
    categorical_columns = ['Пол', 'Область изучения', 'Текущий уровень работы', 'Предпринимательство']
    for col in categorical_columns:
        le = LabelEncoder()
        le.fit(df[col].unique())
        df[col] = le.transform(df[col])
        label_encoders[col] = le
    
    # Исключение столбцов с кластерами
    cluster_columns = [col for col in df.columns if '_cluster' in col or col == 'Cluster']
    X = df.drop(columns=['Идентификатор студента', 'Начальная зарплата', 'Годы до повышения', 
                         'Удовлетворенность карьерой', 'Баланс между работой и личной жизнью'] + cluster_columns)
    feature_columns = X.columns.tolist()  # Сохраняем только релевантные признаки
    y = df[['Начальная зарплата', 'Годы до повышения', 'Удовлетворенность карьерой', 'Баланс между работой и личной жизнью']]
    
    # Нормализация признаков
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    scalers['X'] = scaler_X
    
    # Масштабирование целевых переменных
    target_scalers = {
        'salary': MinMaxScaler(feature_range=(25000, 150000)),
        'promotion': MinMaxScaler(feature_range=(1, 5)),
        'satisfaction': MinMaxScaler(feature_range=(1, 10)),
        'balance': MinMaxScaler(feature_range=(1, 10))
    }
    
    y_scaled = np.hstack((
        target_scalers['salary'].fit_transform(y[['Начальная зарплата']]),
        target_scalers['promotion'].fit_transform(y[['Годы до повышения']]),
        target_scalers['satisfaction'].fit_transform(y[['Удовлетворенность карьерой']]),
        target_scalers['balance'].fit_transform(y[['Баланс между работой и личной жизнью']])
    ))
    
    scalers.update(target_scalers)
    
    return df  # Возвращаем DataFrame для использования в predict

def load_model(model_load_path="model.cbm"):
    """
    Загружает модель из файла.
    """
    global model
    model = CatBoostRegressor()
    model.load_model(model_load_path)
    print(f"Модель загружена из файла: {model_load_path}")

def predict(age, gender, hs_gpa, exam_score, exam_type, uni_rank, uni_gpa, field_of_study, 
           internships, projects, certifications, soft_skills, networking, 
           job_offers, current_job_level, entrepreneurship):
    # Преобразование баллов SAT в баллы ЕГЭ, если выбран SAT
    if exam_type == 'SAT':
        ege_score = sat_to_ege(exam_score)
    elif exam_type == 'ЕГЭ':
        ege_score = exam_score
    else:
        raise ValueError("Тип аттестации должен быть 'SAT' или 'ЕГЭ'.")
    
    # Кодирование входных данных
    input_data = {
        'Возраст': age,
        'Пол': label_encoders['Пол'].transform([gender])[0],
        'Средний балл в школе': hs_gpa,
        'Балл ЕГЭ': ege_score,  # Используем баллы ЕГЭ
        'Рейтинг университета': uni_rank,
        'Средний балл в университете': uni_gpa,
        'Область изучения': label_encoders['Область изучения'].transform([field_of_study])[0],
        'Количество стажировок': internships,
        'Количество проектов': projects,
        'Сертификаты': certifications,
        'Оценка мягких навыков': soft_skills,
        'Оценка сетевого взаимодействия': networking,
        'Предложения о работе': job_offers,
        'Текущий уровень работы': label_encoders['Текущий уровень работы'].transform([current_job_level])[0],
        'Предпринимательство': label_encoders['Предпринимательство'].transform([entrepreneurship])[0]
    }
    
    # Создание входного вектора в правильном порядке
    X_new = np.array([[input_data[col] for col in feature_columns]])
    
    # Нормализация и предсказание
    X_scaled = scalers['X'].transform(X_new)
    predictions = model.predict(X_scaled)
    
    # Обратное преобразование результатов
    salary = scalers['salary'].inverse_transform(predictions[:, 0].reshape(-1, 1))[0][0]
    promotion = scalers['promotion'].inverse_transform(predictions[:, 1].reshape(-1, 1))[0][0]
    satisfaction = scalers['satisfaction'].inverse_transform(predictions[:, 2].reshape(-1, 1))[0][0]
    balance = scalers['balance'].inverse_transform(predictions[:, 3].reshape(-1, 1))[0][0]
    
    # Проверка и ограничение значений в допустимых границах
    salary = max(25000, min(150000, salary))  # Ограничение зарплаты
    promotion = max(1, min(5, promotion))    # Ограничение лет до повышения
    satisfaction = max(1, min(10, satisfaction))  # Ограничение удовлетворенности
    balance = max(1, min(10, balance))       # Ограничение баланса
    
    return {
        'salary': salary,
        'promotion': promotion,
        'satisfaction': satisfaction,
        'balance': balance
    }

if __name__ == "__main__":
    # Загрузка данных для подготовки кодировщиков и масштабирования
    data_path = 'education_career_success_translated.csv'
    load_and_prepare_data(data_path)
    
    # Загрузка модели из файла
    load_model("model.cbm")
    
    # Предсказание параметров
    result = predict(
        age=21,
        gender='Male',
        hs_gpa=3.8,
        exam_score=90,  # Балл SAT или ЕГЭ
        exam_type='ЕГЭ',  # Тип аттестации: 'SAT' или 'ЕГЭ'
        uni_rank=5,
        uni_gpa=3.5,
        field_of_study='Computer Science',
        internships=2,
        projects=1,
        certifications=3,
        soft_skills=8,
        networking=7,
        job_offers=1,
        current_job_level='Entry',
        entrepreneurship='No'
    )
    
    print("Predicted Values:")
    print(f"Starting Salary: ${result['salary']:.2f}")
    print(f"Years to Promotion: {result['promotion']:.1f}")
    print(f"Career Satisfaction: {result['satisfaction']:.1f}/10")
    print(f"Work-Life Balance: {result['balance']:.1f}/10")

Модель загружена из файла: model.cbm
Predicted Values:
Starting Salary: $51225.78
Years to Promotion: 3.2
Career Satisfaction: 6.0/10
Work-Life Balance: 5.3/10


C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt

# Глобальные переменные для хранения состояния
label_encoders = {}
scalers = {}
model = None
feature_columns = []

# Функция для перевода баллов SAT в баллы ЕГЭ
def sat_to_ege(sat_score):
    """
    Переводит баллы SAT (400–1600) в баллы ЕГЭ (40–100).
    """
    sat_min, sat_max = 400, 1600
    ege_min, ege_max = 40, 100
    if not (sat_min <= sat_score <= sat_max):
        raise ValueError(f"Балл SAT должен быть в диапазоне {sat_min}–{sat_max}.")
    ege_score = ege_min + (sat_score - sat_min) * (ege_max - ege_min) / (sat_max - sat_min)
    return round(ege_score, 2)

def load_and_prepare_data(data_path):
    global label_encoders, scalers, feature_columns
    
    # Загрузка данных
    df = pd.read_csv(data_path)
    
    # Преобразование баллов SAT в баллы ЕГЭ
    df['Балл ЕГЭ'] = df['SAT_Score'].apply(sat_to_ege)
    df.drop(columns=['SAT_Score'], inplace=True)  # Удаляем старый столбец "SAT_Score"
    
    # Кодирование категориальных переменных
    categorical_columns = ['Gender', 'Field_of_Study', 'Current_Job_Level', 'Entrepreneurship']
    for col in categorical_columns:
        le = LabelEncoder()
        le.fit(df[col].unique())
        df[col] = le.transform(df[col])
        label_encoders[col] = le
    
    # Исключение столбцов с кластерами
    cluster_columns = [col for col in df.columns if '_cluster' in col or col == 'Cluster']
    X = df.drop(columns=['Student_ID', 'Starting_Salary', 'Career_Satisfaction', 
                         'Years_to_Promotion', 'Work_Life_Balance'] + cluster_columns)
    feature_columns = X.columns.tolist()  # Сохраняем только релевантные признаки
    y = df[['Starting_Salary', 'Years_to_Promotion', 'Career_Satisfaction', 'Work_Life_Balance']]
    
    # Нормализация признаков
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    scalers['X'] = scaler_X
    
    # Масштабирование целевых переменных
    target_scalers = {
        'salary': MinMaxScaler(feature_range=(25000, 150000)),
        'promotion': MinMaxScaler(feature_range=(1, 5)),
        'satisfaction': MinMaxScaler(feature_range=(1, 10)),
        'balance': MinMaxScaler(feature_range=(1, 10))
    }
    
    y_scaled = np.hstack((
        target_scalers['salary'].fit_transform(y[['Starting_Salary']]),
        target_scalers['promotion'].fit_transform(y[['Years_to_Promotion']]),
        target_scalers['satisfaction'].fit_transform(y[['Career_Satisfaction']]),
        target_scalers['balance'].fit_transform(y[['Work_Life_Balance']])
    ))
    
    scalers.update(target_scalers)
    
    return df  # Возвращаем DataFrame для использования в predict

def load_model(model_load_path="model.cbm"):
    """
    Загружает модель из файла.
    """
    global model
    model = CatBoostRegressor()
    model.load_model(model_load_path)
    print(f"Модель загружена из файла: {model_load_path}")

# Функция для вычисления треугольной функции принадлежности
def triangular_membership(x, a, b, c):
    return np.maximum(0, np.minimum((x - a) / (b - a), (c - x) / (c - b)))

def calculate_fuzzy_membership(value, min_val, max_val):
    """
    Вычисляет степень принадлежности значения к нечетким множествам.
    """
    k = (max_val - min_val) / 4
    k1 = min_val + k
    k2 = k1 + k
    k3 = k2 + k
    
    degree_low = triangular_membership(value, min_val, k1, k2)
    degree_medium = triangular_membership(value, k1, k2, k3)
    degree_high = triangular_membership(value, k2, k3, max_val)
    
    # Определение категории
    if degree_low > degree_medium and degree_low > degree_high:
        category = "низкий"
    elif degree_medium > degree_low and degree_medium > degree_high:
        category = "средний"
    else:
        category = "высокий"
    
    return {
        'low': degree_low,
        'medium': degree_medium,
        'high': degree_high,
        'category': category
    }

def predict_for_test_data(test_data_path):
    # Загрузка данных из test.csv
    test_df = pd.read_csv(test_data_path)
    
    # Преобразование баллов SAT в баллы ЕГЭ
    test_df['Балл ЕГЭ'] = test_df['SAT_Score'].apply(sat_to_ege)
    test_df.drop(columns=['SAT_Score'], inplace=True)  # Удаляем старый столбец "SAT_Score"
    
    # Кодирование категориальных переменных
    for col in ['Gender', 'Field_of_Study', 'Current_Job_Level', 'Entrepreneurship']:
        test_df[col] = label_encoders[col].transform(test_df[col])
    
    # Создание входного вектора в правильном порядке
    X_new = test_df[[col for col in feature_columns]].values
    
    # Нормализация и предсказание
    X_scaled = scalers['X'].transform(X_new)
    predictions = model.predict(X_scaled)
    
    # Обратное преобразование результатов
    salaries = scalers['salary'].inverse_transform(predictions[:, 0].reshape(-1, 1)).flatten()
    promotions = scalers['promotion'].inverse_transform(predictions[:, 1].reshape(-1, 1)).flatten()
    satisfactions = scalers['satisfaction'].inverse_transform(predictions[:, 2].reshape(-1, 1)).flatten()
    balances = scalers['balance'].inverse_transform(predictions[:, 3].reshape(-1, 1)).flatten()
    
    # Добавление предсказаний в DataFrame
    test_df['Predicted_Salary'] = salaries
    test_df['Predicted_Years_to_Promotion'] = promotions
    test_df['Predicted_Career_Satisfaction'] = satisfactions
    test_df['Predicted_Work_Life_Balance'] = balances
    
    # Вычисление нечетких множеств для каждого параметра
    parameters = {
        'Predicted_Salary': (25000, 150000),
        'Predicted_Years_to_Promotion': (1, 5),
        'Predicted_Career_Satisfaction': (1, 10),
        'Predicted_Work_Life_Balance': (1, 10)
    }
    
    for param, (min_val, max_val) in parameters.items():
        test_df[f'{param}_Fuzzy'] = test_df[param].apply(
            lambda x: calculate_fuzzy_membership(x, min_val, max_val)
        )
    
    return test_df

if __name__ == "__main__":
    # Загрузка данных для подготовки кодировщиков и масштабирования
    data_path = 'education_career_success.csv'
    load_and_prepare_data(data_path)
    
    # Загрузка модели из файла
    load_model("model.cbm")
    
    # Предсказание для тестового датасета
    test_data_path = 'test.csv'
    results = predict_for_test_data(test_data_path)
    
    # Сохранение результатов в файл
    results.to_csv('test_results_with_fuzzy.csv', index=False)
    print("Предсказания с нечеткими множествами сохранены в файл: test_results_with_fuzzy.csv")

Модель загружена из файла: model.cbm


C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Предсказания с нечеткими множествами сохранены в файл: test_results_with_fuzzy.csv


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from catboost import CatBoostRegressor

# Глобальные переменные для хранения состояния
label_encoders = {}
scalers = {}
model = None
feature_columns = []

# Функция для перевода баллов SAT в баллы ЕГЭ
def sat_to_ege(sat_score):
    """
    Переводит баллы SAT (400–1600) в баллы ЕГЭ (40–100).
    """
    sat_min, sat_max = 400, 1600
    ege_min, ege_max = 40, 100
    if not (sat_min <= sat_score <= sat_max):
        raise ValueError(f"Балл SAT должен быть в диапазоне {sat_min}–{sat_max}.")
    ege_score = ege_min + (sat_score - sat_min) * (ege_max - ege_min) / (sat_max - sat_min)
    return round(ege_score, 2)

def load_and_prepare_data(data_path):
    global label_encoders, scalers, feature_columns
    
    # Загрузка данных
    df = pd.read_csv(data_path)
    
    # Преобразование баллов SAT в баллы ЕГЭ
    df['Балл ЕГЭ'] = df['SAT_Score'].apply(sat_to_ege)
    df.drop(columns=['SAT_Score'], inplace=True)  # Удаляем старый столбец "SAT_Score"
    
    # Кодирование категориальных переменных
    categorical_columns = ['Gender', 'Field_of_Study', 'Current_Job_Level', 'Entrepreneurship']
    for col in categorical_columns:
        le = LabelEncoder()
        le.fit(df[col].unique())
        df[col] = le.transform(df[col])
        label_encoders[col] = le
    
    # Исключение столбцов с кластерами
    cluster_columns = [col for col in df.columns if '_cluster' in col or col == 'Cluster']
    X = df.drop(columns=['Student_ID', 'Starting_Salary', 'Career_Satisfaction', 
                         'Years_to_Promotion', 'Work_Life_Balance'] + cluster_columns)
    feature_columns = X.columns.tolist()  # Сохраняем только релевантные признаки
    y = df[['Starting_Salary', 'Years_to_Promotion', 'Career_Satisfaction', 'Work_Life_Balance']]
    
    # Нормализация признаков
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    scalers['X'] = scaler_X
    
    # Масштабирование целевых переменных
    target_scalers = {
        'salary': MinMaxScaler(feature_range=(25000, 150000)),
        'promotion': MinMaxScaler(feature_range=(1, 5)),
        'satisfaction': MinMaxScaler(feature_range=(1, 10)),
        'balance': MinMaxScaler(feature_range=(1, 10))
    }
    
    y_scaled = np.hstack((
        target_scalers['salary'].fit_transform(y[['Starting_Salary']]),
        target_scalers['promotion'].fit_transform(y[['Years_to_Promotion']]),
        target_scalers['satisfaction'].fit_transform(y[['Career_Satisfaction']]),
        target_scalers['balance'].fit_transform(y[['Work_Life_Balance']])
    ))
    
    scalers.update(target_scalers)
    
    return df  # Возвращаем DataFrame для использования в predict

def load_model(model_load_path="model.cbm"):
    """
    Загружает модель из файла.
    """
    global model
    model = CatBoostRegressor()
    model.load_model(model_load_path)
    print(f"Модель загружена из файла: {model_load_path}")

def calculate_fuzzy_membership(value, min_val, max_val):
    """
    Вычисляет степень принадлежности значения к нечетким множествам.
    """
    k = (max_val - min_val) / 4
    k1 = min_val + k
    k2 = k1 + k
    k3 = k2 + k
    
    degree_low = max(0, min((value - min_val) / (k1 - min_val), (k2 - value) / (k2 - k1)))
    degree_medium = max(0, min((value - k1) / (k2 - k1), (k3 - value) / (k3 - k2)))
    degree_high = max(0, min((value - k2) / (k3 - k2), (max_val - value) / (max_val - k3)))
    
    # Определение категории
    if degree_low > degree_medium and degree_low > degree_high:
        category = "low"
    elif degree_medium > degree_low and degree_medium > degree_high:
        category = "medium"
    else:
        category = "high"
    
    return {
        'low': degree_low,
        'medium': degree_medium,
        'high': degree_high,
        'category': category
    }

def predict_for_test_data(test_data_path):
    # Загрузка данных из test.csv
    test_df = pd.read_csv(test_data_path)
    
    # Преобразование баллов SAT в баллы ЕГЭ
    test_df['Балл ЕГЭ'] = test_df['SAT_Score'].apply(sat_to_ege)
    test_df.drop(columns=['SAT_Score'], inplace=True)  # Удаляем старый столбец "SAT_Score"
    
    # Кодирование категориальных переменных
    for col in ['Gender', 'Field_of_Study', 'Current_Job_Level', 'Entrepreneurship']:
        test_df[col] = label_encoders[col].transform(test_df[col])
    
    # Создание входного вектора в правильном порядке
    X_new = test_df[[col for col in feature_columns]].values
    
    # Нормализация и предсказание
    X_scaled = scalers['X'].transform(X_new)
    predictions = model.predict(X_scaled)
    
    # Обратное преобразование результатов
    salaries = scalers['salary'].inverse_transform(predictions[:, 0].reshape(-1, 1)).flatten()
    promotions = scalers['promotion'].inverse_transform(predictions[:, 1].reshape(-1, 1)).flatten()
    satisfactions = scalers['satisfaction'].inverse_transform(predictions[:, 2].reshape(-1, 1)).flatten()
    balances = scalers['balance'].inverse_transform(predictions[:, 3].reshape(-1, 1)).flatten()
    
    # Добавление предсказаний в DataFrame
    test_df['Predicted_Salary'] = salaries
    test_df['Predicted_Years_to_Promotion'] = promotions
    test_df['Predicted_Career_Satisfaction'] = satisfactions
    test_df['Predicted_Work_Life_Balance'] = balances
    
    # Вычисление нечетких множеств для каждого параметра
    parameters = {
        'Predicted_Salary': (25000, 150000),
        'Predicted_Years_to_Promotion': (1, 5),
        'Predicted_Career_Satisfaction': (1, 10),
        'Predicted_Work_Life_Balance': (1, 10)
    }
    
    for param, (min_val, max_val) in parameters.items():
        test_df[f'{param}_Fuzzy'] = test_df[param].apply(
            lambda x: calculate_fuzzy_membership(x, min_val, max_val)
        )
    
    return test_df

def filter_results_by_user_preferences(df, user_preferences):
    """
    Фильтрует строки на основе предпочтений пользователя.
    """
    mask = True
    for param, category in user_preferences.items():
        mask &= df[f'{param}_Fuzzy'].apply(lambda x: x['category'] == category)
    filtered_df = df[mask]
    return filtered_df

if __name__ == "__main__":
    # Загрузка данных для подготовки кодировщиков и масштабирования
    data_path = 'education_career_success.csv'
    load_and_prepare_data(data_path)
    
    # Загрузка модели из файла
    load_model("model.cbm")
    
    # Предсказание для тестового датасета
    test_data_path = 'test.csv'
    results = predict_for_test_data(test_data_path)
    
    # Сохранение всех данных с предсказанными категориями
    results.to_csv('all_predictions_with_fuzzy.csv', index=False)
    print("Все данные с предсказанными категориями сохранены в файл: all_predictions_with_fuzzy.csv")
    
    # Пользовательские предпочтения
    user_preferences = {
        'Predicted_Salary': 'low',
        'Predicted_Years_to_Promotion': 'low',
        'Predicted_Career_Satisfaction': 'low',
        'Predicted_Work_Life_Balance': 'medium'
    }
    
    # Фильтрация строк по предпочтениям пользователя
    filtered_results = filter_results_by_user_preferences(results, user_preferences)
    
    # Сохранение отфильтрованных данных
    filtered_results.to_csv('filtered_results.csv', index=False)
    print("Отфильтрованные данные сохранены в файл: filtered_results.csv")

Модель загружена из файла: model.cbm


C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Все данные с предсказанными категориями сохранены в файл: all_predictions_with_fuzzy.csv
Отфильтрованные данные сохранены в файл: filtered_results.csv
